In [1]:
import pandas as pd
import ast
from funciones import extract
import pyarrow
import json
import gzip
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
%load_ext memory_profiler
import pyarrow.parquet as pq

In [2]:
df = pd.read_csv(r'../data_set_limpio/games_preparado.csv.gz')

In [3]:
opinion = pd.read_csv(r'../data_set_limpio/reviews_preparado.csv.gz')

In [4]:
item_user = pd.read_csv(r'../data_set_limpio/item_cantidad_usuarios.csv.gz')

In [5]:
def developer(developer: str):
    if developer not in list(df['developer']):
        return {'El desarrollador no se encuentra en la base de datos'}

    fechas = df['release_date'].unique()  # --- > Obtiene las fechas únicas de lanzamiento

    anio = {} 
    free = {} 

    for x in fechas:
        filter_condition = (df['release_date'] == x) & (df['developer'] == developer) 
        developer_releases = df[filter_condition]  # --- > Filtra los lanzamientos del desarrollador

        if len(developer_releases) != 0:
            anio[x] = len(developer_releases)  # --- > Almacena la cantidad de lanzamientos del desarrollador
            free[x] = len(developer_releases[developer_releases['price'] == 0.0])  # --- > Almacena la cantidad de lanzamientos gratuitos

    for x, y in free.items():
        free[x] = f"{round((y / anio[x]) * 100, 2)}%"  # --- > Calcula el porcentaje de lanzamientos gratuitos respecto al total por año

    # --- > Convierte las claves del diccionario 'anio' y 'free' a cadenas
    anio = {str(k): v for k, v in anio.items()}
    free = {str(k): v for k, v in free.items()}

    resultado = {'Cantidad de Items': anio, 'Contenido Free': free} 

    return resultado  # --- > Devuelve el resultado


In [6]:
developer('Valve') # --- > prueba

{'Cantidad de Items': {'2017': 2,
  '1998': 1,
  '2006': 2,
  '2003': 1,
  '2007': 3,
  '2000': 2,
  '2001': 1,
  '2004': 5,
  '2008': 1,
  '2009': 1,
  '1999': 1,
  '2010': 2,
  '2011': 1,
  '2012': 1},
 'Contenido Free': {'2017': '0.0%',
  '1998': '0.0%',
  '2006': '0.0%',
  '2003': '0.0%',
  '2007': '33.33%',
  '2000': '0.0%',
  '2001': '0.0%',
  '2004': '0.0%',
  '2008': '0.0%',
  '2009': '0.0%',
  '1999': '0.0%',
  '2010': '50.0%',
  '2011': '0.0%',
  '2012': '0.0%'}}

In [7]:
def userdata(user: str):
    try:
        if user not in item_user['user_id'].unique():
            return {f'El usuario {user}, no existe.'}  # --- > Verifica si el usuario no existe en la base de datos

        usuario = 0
        chunks = pd.read_csv(r'../data_set_limpio/item_desplegado.csv.gz', chunksize=200000)  # --- > Lee el archivo CSV en bloques para optimizar lo mejor posible el uso de los recursos
        for chunk in chunks:
            if chunk['user'].isin([user]).any():
                usuario = chunk[chunk['user'] == user]  # --- > Obtiene el usuario si se encuentra en el chunk
                break

        resultado = {
            'Usuario': user,
            'Dinero gastado': str(usuario['price'].sum())+'$',  # --- > Calcula el dinero gastado por el usuario
            'cantidad de items': str(item_user.loc[item_user['user_id'] == user]['items_count'].values[0]),  # --- > Obtiene la cantidad de items para el usuario
            'Porcentaje de recomendaciones': str(round((len(opinion.loc[opinion['user'] == user]) / item_user.loc[item_user['user_id'] == user]['items_count'].values[0]) * 100, 2))+'%'  # --- > Calcula el porcentaje de recomendaciones del usuario
        }

        return resultado  # --- > Devuelve el resultado
    except:
        return {'El usuario no se encuentra en la base de datos.'}  # --- > Manejo en caso de errores


In [8]:
userdata('76561197970982479')

{'Usuario': '76561197970982479',
 'Dinero gastado': '2752.41$',
 'cantidad de items': '277',
 'Porcentaje de recomendaciones': '1.08%'}

In [9]:
def best_developer_year(year: int):
    anio = pd.read_csv(r'../data_set_limpio//Max_developer_year.csv')  # --- > Lee el archivo CSV que contiene datos de los mejores desarrolladores por año

    anio = anio.sort_values('Anio', ascending=False)  # --- > Ordena los datos por el año en orden descendente

    seleccion = anio.loc[anio['Anio'] == year].copy()  # --- > Selecciona los datos correspondientes al año ingresado

    seleccion.fillna('Sin informacion', inplace=True)  # --- > Rellena los valores faltantes con 'Sin información'

    if not seleccion['Anio'].empty:
        
        # --- > Si hay datos disponibles para el año proporcionado, se crea un diccionario con la información de los mejores desarrolladores
        
        respuesta = {
            'Anio': str(seleccion['Anio'].values[0]),
            'Top 1': str(seleccion['Top 1'].values[0]),
            'Top 2': str(seleccion['Top 2'].values[0]),
            'Top 3': str(seleccion['Top 3'].values[0])
        }

    else:
        
        # --- > Si no se proporciona un año relevante dentro del rango disponible, se devuelve un mensaje con el rango disponible
        respuesta = {f'No ingreso un valor relevante, este es el rango disponible ({str(anio["Anio"].min())} - {str(anio["Anio"].max())})'}

    return respuesta


In [11]:
best_developer_year(2014)

{'Anio': '2014',
 'Top 1': "('Edge of Reality', 198)",
 'Top 2': "('Coffee Stain Studios', 152)",
 'Top 3': "('New World Interactive', 132)"}

In [26]:
def review_developer(des: str):
    try:
        table = pq.read_table(r'../data_set_limpio/recomends_dev.parquet')  # --- > Lee el archivo Parquet que contiene reseñas de desarrolladores

        dev = table.to_pandas()  # --- > Convierte los datos en un DataFrame de Pandas

        developer = dev.loc[dev['Developers'] == des]  # --- > Filtra las reseñas para el desarrollador especificado

        respuesta = {
            'Deloper': str(developer['Developers'].values[0]),
            'Reviews positivos': str(developer['Positivo'].values[0]),
            'Reviews negativos': str(developer['Negativo'].values[0]) 
        }

        dev = 0  # --- > Reinicializa la variable 'dev' a 0
        return respuesta

    except:
        return {'No ingreso un valor relevante, o el desarrollador no tiene Resenias'}  # --- > Captura excepciones y devuelve un mensaje indicando la falta de valor relevante o reseñas del desarrollador


In [27]:
review_developer('Freejam') # --- > testeo

{'Deloper': 'Freejam', 'Reviews positivos': '211', 'Reviews negativos': '71'}